# Lecture 4

## Approximate median finding and streaming algorithms, part 1

# Approximate median finding

## Mean and median

Recall that the *median* of a collection of numbers is the middle number in sorted order. The *mean* is the average.  The median is more robust, as a single bad value can change the mean by an arbitrary amount but not the median. Note that the median generalizes the idea of order statistic, and everything here can also be used for any oder statistic or to compute quantiles.  

In [23]:
# Returns the mean of a list
def meanOfList(A):
    return sum(A)/len(A) 
# Returns the median of a list
def medianOfList(A):
    Asorted=sorted(A)
    return Asorted[len(A)//2]

A=[1,3,99,32,78,2,4,6,8]
print(A," mean: ",meanOfList(A)," median ",medianOfList(A))
A[0]=10000
print(A," mean: ",meanOfList(A)," median ",medianOfList(A))

[1, 3, 99, 32, 78, 2, 4, 6, 8]  mean:  25.88888888888889  median  6
[10000, 3, 99, 32, 78, 2, 4, 6, 8]  mean:  1136.888888888889  median  8


In this code, mean runs in time $O(n)$ while median uses sorting and runs in time $O(n \log n)$. It is possible to compute the median in linear time. The [statistics](https://docs.python.org/3/library/statistics.html) package has efficient code for mean and median.

Can we do better than linear time? Meaning can we compute the mean or the median without looking at all the data? The answer is no. For the mean we can no even approximate it without looking at all the values, as the one value we miss could change the mean in an unrestricted way.

However, there is hope for the median. Let us define formally what a $\epsilon$-approximate median is:

Given a list of size $n$, $x$ is a $\epsilon$-approximate median is any value from $\frac{(1-\epsilon)n}{2}$nd to the $\frac{(1+\epsilon)n}{2}$rd items in the list.

So a $50\%$-approximate median would be an item that would be in the middle half of the list, if sorted.

Here is a very simple algorithm to return a $50\%$-approximate median that works 50% of the time: pick a random element and return it. This only looks at one element.

But the 50% failure probability is quite high. Can we design a method that looks at more than 1, but less than all of the items and has a lower failure probability?

A simple idea is this: pick $k$ items at random and return the median of them

In [24]:
import random 
def approxMedianOfList(A,k):
    kRandom=random.choices(A,k=k)
    kRandom.sort()
    return kRandom[len(kRandom)//2]

A=[random.randrange(1,1000)**2 for i in range(10000)]
for k in range(1,100,10):
    print("Approx median with k=",k,":",approxMedianOfList(A,k))
print("Real median",medianOfList(A))
print("Real mean",meanOfList(A))

Approx median with k= 1 : 817216
Approx median with k= 11 : 150544
Approx median with k= 21 : 227529
Approx median with k= 31 : 271441
Approx median with k= 41 : 167281
Approx median with k= 51 : 299209
Approx median with k= 61 : 211600
Approx median with k= 71 : 287296
Approx median with k= 81 : 205209
Approx median with k= 91 : 272484
Real median 254016
Real mean 335953.9648


Ok, so how good is this? Once again we turn to Chernoff. Let us first bound the probability that the approximate median is too high.

Let $X_i$ be the indicator random variable that is 1 iff the $i$th random value is not an $\epsilon$-approximate median because it is too high, call this a high sample. We know $E[X_i]=\frac{1-\epsilon}{2}$. 

We know the median-of-medians is to high if at least half of the samples are high. Let $X$ be the total number of high samples, $\sum_{i=1}^k X_i$. We want to bound

$$ Pr[X \geq \frac{1}{2}k] $$

Chernoff wants to see something like

$$Pr[X \geq (1+\delta) E[X] ]$$

As we know $E[x]= k \cdot \frac{1-\epsilon}{2}$, we solve for $\delta$:

$$(1+\delta)\left( k \cdot \frac{1-\epsilon}{2}\right) = \frac{1}{2}k
$$

Thus $\delta= \frac{1}{1-\epsilon}-1$, which is at most 1. Thus we can use Chernoff as follows:

$\begin{align*}
Pr[X \geq (1+\delta) E[X] ] &\leq e^{-\frac{\delta^2E[X]}{2+\delta}}
\\
&\leq e^{-\frac{\delta^2E[X]}{3}}
\\
&\leq e^{-\frac{\left(\frac{1}{1-\epsilon}-1\right)^2 k \cdot \frac{1-\epsilon}{2}}{3}}
\\
&= e^{ -k \frac{\epsilon^2}{6-6\epsilon} }
\\
&\leq  e^{ -\frac{k\epsilon^2}{6}}
\end{align*}
$

Remember, this was the chance of failure because there were to many samples that were too large. The chance of failure because there were too many samples that are too small is the same. Thus, by the union bound, the total chance of failure is at most $ 2  e^{ -\frac{k\epsilon^2}{6}}$.

How should be choose $k$ to get a certain error rate, call it $\gamma$, for a $\epsilon$-approximate median? Solve:

$$
\begin{align*}
\gamma &= 2  e^{ -\frac{k\epsilon^2}{6}} \\
\gamma &= 2  e^{ -\frac{k\epsilon^2}{6}} \\ 
\ln \frac{\gamma}{2} &= -\frac{k\epsilon^2}{6}\\
\frac{6}{\epsilon^2} \ln \frac{\gamma}{2}  &= - k  \\
\frac{6}{\epsilon^2} \ln \frac{2}{\gamma}  &=  k  \\
\end{align*}
$$

So, if we wanted to get a 10%-approximate median with a failure rate of $0.01\%$, we set $\epsilon=0.1$ and $\gamma=0.0001$ which gives $\gamma=5942$. Around 6000 may seem like a lot, but when you have millions or billions of data items, this is a real improvement.

# Streaming

A streaming algorithm is one that looks at the data once, one element at a time. The idea is that it does not store the data but stores some very small amount of data that allows it to compute what it wants. 

Streaming algorithms are of use not only in their original context where data is going by so quickly that you can not store it, but also in the big data context where you have a data set that is so big that you don't want to look at it more than once and copying or moving it is out of the question

## Streaming and iterators

In python (and other similar languages) the construct `for x in X` works like a stream, where the data is not stored and you can look at each item in `X` once. How do you create such a stream? One way is to use what python calls a generator, where each element of the stream is produced with a `yield` statement. Execution then alternates between the `for` loop and the generator, and nothing is stored.

In [25]:
# Streaming

import random

def someRandom(maxcount,maxnum,debug=False):
    for i in range(random.randint(1,maxcount)):
        if debug:
            print("SomeRandom")
        yield(random.randint(0,maxnum))

In [26]:
# Good code to compute the average
# See how execution alternates beck and forth from the generator
# And nothing is stored
l=0
s=0
for x in someRandom(10,1000,debug=True):
    print("Average")
    s+=x
    l+=1
print(s/l)

SomeRandom
Average
SomeRandom
Average
SomeRandom
Average
SomeRandom
Average
457.75


In [27]:
# Bad code to compute the average
# All values are saved in A
A=[x for x in someRandom(10,1000,debug=True)]
print("A created")
print(sum(A)/len(A))


SomeRandom
SomeRandom
SomeRandom
A created
675.6666666666666


# Median streaming

In order to compute the median in the streaming model, you can use exactly the same sampling method we discussed above. The only question is how to get a uniform sample in the streaming model. The answer given a stream of data $x_1, \ldots x_n$, at step $i$ replace the current sample with $x_i$ with probability $\frac{1}{i}$. Thus the chance that $x_i$ is chosen, is the chance that $x_i$ is picked and the chance that $x_j$, $j>i$ are all not picked.

$$
\displaystyle \frac{1}{i} \prod_{j={i+1}}^n \frac{j-1}{j}
$$

This telescopes to $\frac{1}{n}$:

$$
\displaystyle  \frac{\prod_{j={i+1}}^n (j-1)}{i\prod_{j={i+1}}^n j}
=
\displaystyle  \frac{\prod_{j={i}}^{n-1} j}{n\prod_{j={i}}^{n-1} j}
=\frac{1}{n}$$







# Distinct elements

Notation:

- Let $d$ be the number of distinct elements. Name them $x_1,\ldots x_d$.
- Let $z(x_i)$ be the number of trailing zeros in the binary representation of $h(x_i)$
- Let $z = max_i z(x_i)$. This is the maximum number of zeros
- Let $\hat{d} = 2^{z+\frac{1}{2}}$. This is the output approximation for $d$.

What we want to prove is that usually $\hat{d}$ is close to $d$. Specifically we will bound that they are within a factor of 3 of each other.

Let us proceed with the upper bound. We want to ask, what is the chance that $\hat{d}\geq 3d$. As usual we want to break things down into indicator random variables so that we can use linearity of expectation.

We define the indicator random variable $z_j(x_i)$ which is true if 
$z(x_i)\leq j$, and let $Y_j = \sum_{i}^d z_j(x_i)$. Thus $Y_j$ is the number of distinct items in the stream that have zeros values at least $j$. Thus if $z$, the maximum number of zeros is $j$, we known $Y_j \geq 1$ and $Y_{j+1}=0$. Thus we can relate the maximum with the sum, which we know how to deal with.

Let's look at the chance that $\hat{d}$ is too big by a factor of three.

$$
\begin{align*}
&P[\hat{d} \geq 3d] \\
&= P[ 2^{z+\frac{1}{2}} \geq 2^{\log (3d)} ]
\\
&= P[ z+\frac{1}{2} \geq \log (3d)]
\\
&= P[Y_{\log (3d)- \frac{1}{2}} \geq 1]
\\
&\leq  \frac{E[Y_{\log (3d)- \frac{1}{2}}]}{1}
& \text{Markov: }Pr[X> a ] \leq \frac{E[X]}{k}
\\
& =  E[\sum_{i}^d z_{\log (3d)- \frac{1}{2}}(x_i) ]
&\text{As }Y_j = \sum_{i}^d z_j(x_i)
\\
& =  \sum_{i}^d E[z_{\log (3d)- \frac{1}{2}}(x_i) ]
\\
& =  \sum_{i}^d \frac{1}{2^{\log (3d)- \frac{1}{2} }}
\\
& =  \sum_{i}^d \frac{\sqrt{2}}{3d} 
\\
& =  \frac{\sqrt{2}}{3}\approx 47\%
\end{align*}
$$

Next, Let's look at the chance that $\hat{d}$ is too small. *We did not do the math for this in class, and it is a little different than the too big case*.
As Markov is useless in this case, we introduce the third commonly used inequality:
Chebyshev, which bounds how likely a random variable can be far from its expected value as a function of its variance: 

$$ Pr[ | X - E[X] | \geq k ] \leq \frac{Var[X]}{k^2}$$

The variance $Var[X]$ is has the easy-to-use formula of $E[X]^2-E[X^2]$. This is very easy to use for indicator random variables, as these are always 0 or 1, which are the two numbers that don't change when you square them; this for indicator random variables $E[X^2]$ is $E[X]$. Finally, for independent variables $Var[X+Y]=Var[X]+Var[Y]$

Ok, so lets look at the chance that $\hat{d}$ is too small by a factor of 3.


$$
\begin{align*}
&P[\hat{d} \leq \frac{d}{3}] \\
&= P[ 2^{z+\frac{1}{2}} < 2^{\log \frac{d}{3}} ]
\\
&= P[ z+\frac{1}{2} < \log \frac{d}{3}]
\\
&= P[Y_{\log \frac{d}{3}+ \frac{1}{2}} = 0 ]
\\
&\leq  P[ | Y_{\log \frac{d}{3}+ \frac{1}{2}} - E[Y_{\log \frac{d}{3}+ \frac{1}{2}}]|  \geq  E[Y_{\log \frac{d}{3}+ \frac{1}{2}}] ]
\\
&\leq  \frac{Var[Y_{\log \frac{d}{3}+ \frac{1}{2}}]}{E[Y_{\log \frac{d}{3}+ \frac{1}{2}}]^2}
& \text{Chebyshev: }Pr[ | X - E[X] | \geq k ] \leq \frac{Var[X]}{k^2}
\\
&\leq  \frac{E[Y_{\log \frac{d}{3}+ \frac{1}{2}}]}{E[Y_{\log \frac{d}{3}+ \frac{1}{2}}]^2}
& \text{*}
\\
&\leq  \frac{1}{E[Y_{\log \frac{d}{3}+ \frac{1}{2}}]}
\\
&\leq  \frac{\sqrt{2}}{3}
&\text{As  above, $E[Y_m] = \frac{d}{2^m}$}
\end{align*}
$$

Point * is because $Var[Y_m]=Var[\sum_{i}^d z_j(x_i)]=\sum_{i}^d Var [z_m(x_i)] =
\sum_{i}^d (E[ z_m(x_i)^2]-E[ z_m(x_i)]^2 )\leq \sum_{i}^d E[ z_m(x_i)^2]= \sum_{i}^d E[ z_m(x_i)] = E[\sum_{i}^d z_m(x_i)] = E[Y_m]
$

So, we have the probability that $\hat{d}$ is within a factor of 3 of the real value of $d$ is $1-\frac{2 \sqrt{2}}{3}\approx 0.057$.
A 6% success rate is hardly impressive. But, by using several independent computations of the distinct elements, and taking the median value, we can boost the success rate. This is exactly the same as the approximate median finding algorithm with $\epsilon 1-\frac{2 \sqrt{2}}{3}\approx 0.057$. 


In [28]:
import os
import hashlib
import time
import random


seed=str(time.time()).encode('utf-8')

def hash(s):
    ho = hashlib.sha3_256()
    ho.update(s.encode())
    ho.update(seed)
    return int.from_bytes(ho.digest(),"big")

def zeros(x):
    answer=0
    while (True):
        if x & 1<<answer:
            return answer
        answer+=1

def distinctElements(strings):
    S=set()
    for string in strings:
        S.add(string)
    return len(S)
    

def maxZeros(strings):
    return max(zeros(hash(string)) for string in strings )

def approxDistinctElements(strings):
    return 2**(maxZeros(strings)+0.5)
        
        
def randomStringOfChars(length):
    letters = "qwertyuiopasdfghjklzxcvbnnm"
    return "".join((random.choice(letters) for i in range(length)))


A=[randomStringOfChars(50) for s in range(10000)]
print(distinctElements(A*5)) #Five copies of A
print(maxZeros(A))
print(approxDistinctElements(A))

10000
10
1448.1546878700494


# Homework

In order to get a uniform sample in the streaming model, we needed to take the $i$th item with probability $\frac{1}{i}$. This involves computing a random number. 

Recall that to compute 10%-approximate median with a failure rate of $0.01\%$ we needed about 6000 random samples. Implemented in a straightforward way, this would mean 6000 random numbers would be needed for each data item, which is a lot!

So your homework is to create an method to compute a sample in a stream that is *almost* uniform, that is that the probability to pick a given item is within a factor of two of $\frac{1}{n}$ if the stream has had $n$ items so far. Your method instead of having $n$ random numbers generated in total should generate far fewer: a number logarithmic in $n$.

Present your method, code it, and prove that it works.
